In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==2.12.0

#Unzip folder

In [ ]:
!unzip 'images.zip' -d images/

#Import Libraries

In [ ]:
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


#Get Data

In [ ]:
# Crear las listas archivos de audio y su etiqueta correspondiente

from tensorflow.keras.utils import to_categorical
import glob
import os

def getExamples(datafolder):
  X_Image = []
  Y_Classification = []

  # Clasificaciones de clases
  image_classes = [os.path.basename(x) for x in glob.glob(datafolder + '*')]

  for i, image_class in enumerate(image_classes):
    for file in glob.glob(os.path.join(datafolder, image_class) + '/*.jpg'):
      X_Image.append(file)
      Y_Classification.append(np.array(to_categorical(i, num_classes=len(image_classes)),dtype=np.float32))
  return np.asarray(X_Image), np.asarray(Y_Classification)

In [ ]:
datafolder= '/content/images/'
X_Image, Y_Class = getExamples(datafolder)
print(len(X_Image), len(Y_Class))

In [ ]:
print(X_Image[300])
print(Y_Class[300])

In [ ]:
from sklearn.model_selection import train_test_split

X_Image, X_Image_test, Y_Class, Y_Class_test = train_test_split(X_Image, Y_Class, test_size=0.25)
print(len(X_Image))
print(len(Y_Class))

#Convert file paths to tensors

In [ ]:
def loadExample(example):
  # Cargar la imagen
  img = tf.io.read_file(example)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [48,64], preserve_aspect_ratio=True)
  img = tf.image.rgb_to_grayscale(img);
  img = tf.image.convert_image_dtype(img, dtype=tf.float32)
  #img = img/255.0
  img = tf.reshape(img, [img.shape[0], img.shape[1], 1]);

  return img

In [ ]:
s = loadExample(X_Image[0])
s.shape

In [ ]:
# Implementar el generador de Datos

class MySequence(tf.keras.utils.Sequence):

  def __init__(self, x_image, y_class, batch_size):
    self.x_image = x_image
    self.y_class = y_class
    self.batch_size = batch_size

  def __len__(self):
    return len(self.x_image)//self.batch_size

  def __getitem__(self, idx):

    batch_y = self.y_class[idx * self.batch_size : (idx+1)*self.batch_size]
    batch_x = np.zeros((self.batch_size, s.shape[0], s.shape[1], s.shape[2]), dtype=np.float32)
    for i in range(0, self.batch_size):
      batch_x[i] = loadExample(self.x_image[idx * self.batch_size + i])
    return batch_x, batch_y

In [ ]:
# Verificar la forma de los datos de entrada y la salida esperada

mS=MySequence(X_Image, Y_Class, 32)
my_data=iter(mS)
bx, by = next(my_data)
print(bx.shape, by.shape)

#Import Keras Libraries

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Ad
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten
from tensorflow import keras

#Create Model

In [ ]:

input_tensor = Input(shape=(48, 64,1))
x = Conv2D(32, 3, activation='relu')(input_tensor)
x = MaxPooling2D()(x)
x = Conv2D(32, 3, activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(2, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])



#Watch Graph

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, to_file="model.png", show_shapes=True)

#Training

In [ ]:
batch_size = 16
epochs = 1
h = model.fit(MySequence(X_Image, Y_Class, batch_size),
              shuffle=True,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=MySequence(X_Image_test, Y_Class_test, batch_size))

In [ ]:
def plotExample(example):
  # Cargar la imagen
  img = tf.io.read_file(example)
  img = tf.image.decode_jpeg(img, channels=3)
  plt.imshow(img)

#Make Prediction

In [ ]:
i=5
print(Y_Class_test[i])
print(X_Image_test[i])

plotExample(X_Image_test[i])
tensor = loadExample(X_Image_test[i])
tensor = tf.reshape(tensor, shape=[1,48,64,1])

model.predict(tensor)

In [ ]:
model.save('/content/modelAudioRecognition.h5')

In [ ]:
from keras.models import load_model
smodel=load_model('/content/modelAudioRecognition.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(smodel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('audio_cnn.tflite', 'wb') as f:
  f.write(tflite_model)